## 避免死锁

In [2]:
import threading
from contextlib import contextmanager

# 线程本地状态到已获取的锁的存储信息
_local = threading.local()

# 管理上下文
@contextmanager
def acquire(*locks):
    # 按对象标识符排序锁
    locks = sorted(locks, key=lambda x: id(x))

    # 确保不违反以前获取的锁的锁定顺序
    acquired = getattr(_local,'acquired',[])
    if acquired and max(id(lock) for lock in acquired) >= id(locks[0]):
        raise RuntimeError('Lock Order Violation')

    # 获取所有锁
    acquired.extend(locks)
    _local.acquired = acquired

    try:
        for lock in locks:
            lock.acquire()
        yield
    finally:
        # 以与采集相反的顺序释放锁定
        for lock in reversed(locks):
            lock.release()
        del acquired[-len(locks):]

In [2]:
import threading
x_lock = threading.Lock()
y_lock = threading.Lock()

def thread_1():
    while True:
        with acquire(x_lock, y_lock):
            print('Thread-1')

def thread_2():
    while True:
        with acquire(y_lock, x_lock):
            print('Thread-2')

# t1 = threading.Thread(target=thread_1)
# t1.daemon = True
# t1.start()
# 
# t2 = threading.Thread(target=thread_2)
# t2.daemon = True
# t2.start()


Thread-1
Thread-1
Thread-1
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2


## 死锁避免

In [3]:
import threading

# The philosopher thread
def philosopher(left, right):
    while True:
        with acquire(left,right):
             print(threading.currentThread(), 'eating')

# The chopsticks (represented by locks)
NSTICKS = 5
chopsticks = [threading.Lock() for n in range(NSTICKS)]

# Create all of the philosophers
for n in range(NSTICKS):
    t = threading.Thread(target=philosopher,
                         args=(chopsticks[n],chopsticks[(n+1) % NSTICKS]))
    t.start()


<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 139663234803456)> eating
<Thread(Thread-9, started 13966